## pg_restore cipsrv_nhdpluswshd_m

In [ ]:
import os,sys,psycopg2,argparse;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;
from contextlib import closing;
parser = argparse.ArgumentParser();
parser.add_argument("--mrws_dumpfile"    ,required=False);
parser.add_argument("--use_existing"     ,required=False,default=False,action=argparse.BooleanOptionalAction);
parser.add_argument("--override_username",required=False);
args, _ = parser.parse_known_args();

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

s3bk = os.environ['S3_BUCKET'];
# override: s3bk = 'https://edap-ow-data-commons.s3.amazonaws.com/data/';
lddk = os.path.join(os.path.expanduser('~'),'loading_dock');
dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv';
pswd = os.environ['POSTGRESQL_CIP_PASS'];
thds = 2;

component_name = 'cipsrv_nhdpluswshd_m';
component_type = 'dataset';
if args.mrws_dumpfile is None:
   src = 'cipsrv_nhdpluswshd_m_v21_1.dmp';
else:
   src = args.mrws_dumpfile;

print("source: " + s3bk + src);

In [ ]:
if args.use_existing:
    print("using existing " + src);

else:
    z = common.download(
        s3  = s3bk
       ,src = src
       ,ld  = lddk
       ,trg = src
    );

    if not os.path.exists(os.path.join(lddk,src)):
        raise Exception('failed to download dmp file');

    print("successfully downloaded " + src);
    

In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pswd
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
except:
    raise Exception("database connection error");

print("database is ready");

In [ ]:
with closing(conn.cursor()) as cursor:
    cursor.execute("DROP TABLE IF EXISTS cipsrv_nhdpluswshd_m.catchment_watershed CASCADE");
    conn.commit();

print("preexisting data purged.");

In [ ]:
z = common.pg_restore(
     host
    ,port
    ,dbse
    ,user
    ,pswd
    ,os.path.join(lddk,src)
    ,thds
);

print("pg_restore complete.");

In [ ]:
if z[0] == b'':
    a = common.upsert_registry(conn,component_name,component_type,src,override_username);
else:
    print(str(z));
    sys.exit(-10);
